In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import all the tools we need


# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

# Import the Progress bar TQDM
from tqdm import tqdm


In [ ]:
data=pd.read_excel('/kaggle/input/sentences/Bangla Transformation of Sentence Dataset(BTSD).xlsx')
data.columns=['text','label']
data['num_label']=data['label'].replace({"Simple":0,"Complex":1,"Compound":2})

In [ ]:
data.num_label.value_counts().plot(kind='bar', cmap='viridis')

In [ ]:
cus_data = data.copy()
cus_data.head(3)

In [ ]:
cus = pd.read_csv('/kaggle/input/sentences/all_new.csv')
cus.head(3)

In [ ]:
cor_label = cus.Corrected_Label
cor_text = cus.Text

In [ ]:
pairs = [(a, b) for a, b in zip(cor_label, cor_text)]

In [ ]:
count = 0
for i in tqdm(range(len(pairs))):
    for index, row in cus_data.iterrows():
        if row['text'] == pairs[i][1]:
            count += 1
            cus_data.at[index, 'num_label'] = pairs[i][0]

In [ ]:
print('Total Matched: ', count)

In [ ]:
main_data = data.copy()
data = cus_data.copy()

In [ ]:
cheng = ['তিনি খুব তাড়াতাড়ি বিছানায় গিয়েছিলেন কারণ তার খুব ভোরের ফ্লাইট ছিল', 
         'তিনি তার ক্রিয়াকলাপের প্রভাব উপলব্ধি করার পরে প্রচুর পরিমাণে ক্ষমা চেয়েছিলেন',
         'তিনি ক্লাবে যোগ দিয়েছিলেন কারণ তিনি নতুন লোকের সাথে দেখা করতে চেয়েছিলেন',
         'একটি ওয়েবসাইট তৈরি করা হলে তখন অনেক কাজের মত মনে হয়' ,
         'বৃষ্টি হলে রাস্তা পিচ্ছিল হয়ে যায়।']
chengl = [1, 0, 1, 0, 0]
for i in range(5):
    vtc = cheng[i]
    if vtc in data['text'].values:
        data.loc[data['text'] == vtc, 'num_label'] = chengl[i]

value_to_check =  'ট্রয় একটি বিশাল কুকি খেয়েছে। তার পেটে ব্যথা হয়েছে'
if value_to_check in data['text'].values:
    data.loc[data['text'] == value_to_check, 'text'] = 'ট্রয় একটি বিশাল কুকি খেয়েছে, তার পেটে ব্যথা হয়েছে'

In [ ]:
data.head(2)

In [ ]:
data.num_label.value_counts()

In [ ]:
data.to_csv('/kaggle/working/final_data.csv', index=False)

In [ ]:
emmbed_dict = {}
with open('/kaggle/input/bangla-glove-vectors/bn_glove.39M.300d/bn_glove.39M.300d.txt','r') as f:
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],'float32')
        emmbed_dict[word]=vector

In [ ]:
emmbed_dict['তুমি']

### Create Tokenization

In [ ]:
!pip install -U bnlp_toolkit
!pip install bnlp_toolkit[fasttext]

In [ ]:
from bnlp import BengaliWord2Vec
from bnlp import NLTKTokenizer

bnltk = NLTKTokenizer()

In [ ]:
index = np.random.randint(0, 3000)
# Sample text
text = data['text'][index]

# Tokenize the text
input_tokens = bnltk.word_tokenize(text)

# Print the tokens
print(input_tokens)


In [ ]:
def banglaToken(text):
    vec=[]
    input_token=bnltk.word_tokenize(text)
    for word in input_token:
#         print(word)
        try:
            vec.append(emmbed_dict[word])
#             print(emmbed_dict[word])
        except:
            pass
    return np.array(np.sum(vec, axis=0))

In [ ]:
x = data['text'][0]
x

In [ ]:
# banglaToken(x)

In [ ]:
# emmbed_dict['সন্ধ্যায়']

In [ ]:
index = np.random.randint(0, 3000)
# Sample text
text = data['text'][index]
banglaToken(text)

In [ ]:
tqdm.pandas()
data['vector'] = data.text.progress_apply(banglaToken)

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
data['vector'][188].shape

In [ ]:
banglaToken(data['text'][1885])

In [ ]:
shapes = []
for i in range(len(data)):
    try:
        shapes.append(data['vector'][i].shape)
    except:
        print(data['vector'][i])
        print(i)

In [ ]:
max(shapes)

In [ ]:
data = data.drop(1885)

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
# Set data 
X = list(data['vector'])
y = data.num_label.values

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the lengths of the sets
print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))


In [ ]:
# from sklearn.model_selection import train_test_split
# X = data.vector
# y = data.nlabel

# # Split the data into training (70%), validation (15%), and test (15%) sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Print the lengths of the sets
# print("Training set size:", len(X_train))
# print("Validation set size:", len(X_valid))
# print("Test set size:", len(X_test))

In [ ]:
# X.head()

In [ ]:
# y.head()

In [ ]:
y_train[:2]

In [ ]:
X_train[0]

### Train the Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb


In [ ]:
print(type(X_train))
type(y_train)

In [ ]:
# X_train[0]

In [ ]:
# Import logistic regression
from sklearn.linear_model import LogisticRegression


# Create and train the logistic regression model
LRmodel = LogisticRegression()
LRmodel.fit(list(X_train), y_train)

In [ ]:
# Make predictions in test Set
preds = LRmodel.predict(X_test)

#Calculate accuracy score
accuracy = accuracy_score(y_test, preds)

print(f"Accuracy: {accuracy*100:.2f}", )

### CONFUSION MATRIX

### Test set

In [ ]:
from sklearn import metrics
con_mat = metrics.confusion_matrix(y_test, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, preds, output_dict=True)

cr_df = pd.DataFrame(report).transpose()
cr_df = cr_df.round(2)
cr_df

### Validation set

In [ ]:
# Make predictions in validation set
preds = LRmodel.predict(X_valid)

#Calculate accuracy score
accuracy = accuracy_score(y_valid, preds)

print(f"Accuracy: {accuracy*100:.2f}", )
con_mat = metrics.confusion_matrix(y_valid, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()

## RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

In [ ]:
# Make predictions in test set
preds = rfc.predict(X_test)

#Calculate accuracy score
accuracy = accuracy_score(y_test, preds)

print(f"Accuracy: {accuracy*100:.2f}", )
con_mat = metrics.confusion_matrix(y_test, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()

In [ ]:
# Validation set
# Make predictions in Validation Set
preds = rfc.predict(X_valid)

#Calculate accuracy score
accuracy = accuracy_score(y_valid, preds)

cm = metrics.confusion_matrix(y_valid, preds)
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1,2])
cm_dis.plot()
plt.show()
print(f"Accuracy: {accuracy*100:.2f}", )

## XGBOOST CLASSIFIER

In [ ]:
import xgboost as xgb
XGBmodel=xgb.XGBClassifier(n_estimators=10, max_depth=9, learning_rate=.0005)
XGBmodel.fit(X_train,y_train)
y_pred=XGBmodel.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

In [ ]:
# Make predictions in test set
preds = XGBmodel.predict(X_test)

#Calculate accuracy score
accuracy = accuracy_score(y_test, preds)

con_mat = metrics.confusion_matrix(y_test, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()


print(f"Accuracy: {accuracy*100:.2f}", )

In [ ]:
# Validation set
# Make predictions in Validation Set
preds = XGBmodel.predict(X_valid)

#Calculate accuracy score
accuracy = accuracy_score(y_valid, preds)

cm = metrics.confusion_matrix(y_valid, preds)
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1,2])
cm_dis.plot()
plt.show()
print(f"Accuracy: {accuracy*100:.2f}", )

## CAT BOOST CLASSIFIER

In [ ]:
catboost_params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'depth': 6,
    'l2_leaf_reg': 3,
    'random_seed': 42,
    'verbose': 0
}

In [ ]:
from catboost import CatBoostClassifier
from IPython.display import clear_output

CATmodel = CatBoostClassifier(random_state=71)
CATmodel.fit(X_train, y_train)
clear_output()

In [ ]:
# Make predictions in test set
preds = CATmodel.predict(X_test)

#Calculate accuracy score
accuracy = accuracy_score(y_test, preds)

con_mat = metrics.confusion_matrix(y_test, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()


print(f"Accuracy: {accuracy*100:.2f}", )

In [ ]:
# Make predictions in valid set
preds = CATmodel.predict(X_valid)

#Calculate accuracy score
accuracy = accuracy_score(y_valid, preds)

con_mat = metrics.confusion_matrix(y_valid, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()


print(f"Accuracy: {accuracy*100:.2f}", )

## SVC

In [ ]:
from sklearn.svm import SVC
# Create and train an SVM Classifier
svm_model = SVC(C=10.0, kernel='rbf', degree=3, gamma='scale',random_state=42)
svm_model.fit(X_train, y_train)

In [ ]:
# Make predictions in test set
preds = svm_model.predict(X_test)

#Calculate accuracy score
accuracy = accuracy_score(y_test, preds)

con_mat = metrics.confusion_matrix(y_test, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()


print(f"Accuracy: {accuracy*100:.2f}", )

In [ ]:
# Make predictions in valid set
preds = svm_model.predict(X_valid)

#Calculate accuracy score
accuracy = accuracy_score(y_valid, preds)

con_mat = metrics.confusion_matrix(y_valid, preds)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = con_mat, display_labels=[0,1,2])
cm_display.plot()
plt.show()


print(f"Accuracy: {accuracy*100:.2f}", )

In [ ]:
report = classification_report(y_valid, preds, output_dict=True)

cr_df = pd.DataFrame(report).transpose()
cr_df = cr_df.round(2)
cr_df